# Bathroom Stall

`1` shared resource is available to `N` people. Each person can occupy the resource for some units of time. When a person leaves the resource, the resource is available again.

![alt text](assets/bathroom.png)

## Solution with Lock

We want to make sure no one walks in on someone else.

In [64]:
from threading import Lock, Thread
import time
import random

class BathroomStall:
    def __init__(self):
        self.lock = Lock()

    def use_stall(self, person_id):
        print(f"Person {person_id} is waiting for the bathroom stall.")
        self.lock.acquire()
        print(f"Person {person_id} enters the bathroom stall.")
        time.sleep(random.uniform(0, 1))  # Simulating using the stall
        print(f"Person {person_id} leaves the bathroom stall.")
        self.lock.release()

def use_bathroom(person_id, bathroom: BathroomStall):
    bathroom.use_stall(person_id)
num_people = 5
bathroom = BathroomStall()
threads = [Thread(target=use_bathroom, args=(i, bathroom)) for i in range(num_people)]

for thread in threads:
    thread.start()

for thread in threads:
    thread.join()

print("All people have used the bathroom.")


Person 0 is waiting for the bathroom stall.
Person 0 enters the bathroom stall.
Person 1 is waiting for the bathroom stall.
Person 2 is waiting for the bathroom stall.
Person 3 is waiting for the bathroom stall.
Person 4 is waiting for the bathroom stall.
Person 0 leaves the bathroom stall.
Person 1 enters the bathroom stall.
Person 1 leaves the bathroom stall.
Person 2 enters the bathroom stall.
Person 2 leaves the bathroom stall.
Person 3 enters the bathroom stall.
Person 3 leaves the bathroom stall.
Person 4 enters the bathroom stall.
Person 4 leaves the bathroom stall.
All people have used the bathroom.


# ILLINI Cheer

A cheer is to be orchestrated by `N` sections, one for each unique letter in the cheer.

![alt text](assets/cheer.png)

## Solution with Condition Variables

The conditions will be the state of the cheer

In [68]:
from threading import Thread, Condition

class Cheerleader:
    def __init__(self, cheer):
        self.cheer = cheer
        self.n = len(self.cheer)
        self.current_index = 0
        self.condition = Condition()
    
    def get(self):
        return self.cheer[self.current_index] if self.current_index < len(self.cheer) else None

    def cheer_letter(self, letter):
        last_index_of_section = self.n - list(reversed(self.cheer)).index(letter)
        while self.current_index < last_index_of_section:
            # with/while is used to ensure that the condition is released when the block is exited
            with self.condition:
                while self.get() != letter:
                    self.condition.wait()
                print(letter, end="")
                self.current_index += 1
                self.condition.notify_all()

cheerleader = Cheerleader(cheer="ILLINI")

letters = list(set(cheerleader.cheer))
print(f"Cheer sections: {letters}")

threads = [Thread(target=cheerleader.cheer_letter, args=(section)) for section in letters]
for thread in threads:
    thread.start()

for thread in threads:
    thread.join()
print("\nThe cheer is complete!")


Cheer sections: ['I', 'N', 'L']
ILLINI
The cheer is complete!
